In [3]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [45]:
x, y_ = zip(*[(tX[i], y[i]) for i in range(tX.shape[0]) if tX[i,22] >1 ])
x, y_ = np.array(x), np.array(y_)

In [41]:
y_

array([ 1., -1.,  1., ..., -1.,  1., -1.])

In [42]:
x[:,22]

array([2., 3., 2., ..., 3., 3., 2.])

In [34]:
# Linear regression using gradient descent

initial_w = np.zeros(tX.shape[1])
max_iters = 150
gamma = 0.085

weights, loss = least_squares_GD(y_, standardize(x), initial_w, max_iters, gamma)

Gradient Descent(0/149): loss=0.5
Gradient Descent(1/149): loss=0.45612021505677264
Gradient Descent(2/149): loss=0.43855005949206494
Gradient Descent(3/149): loss=0.4311141816149872
Gradient Descent(4/149): loss=0.4276198069177341
Gradient Descent(5/149): loss=0.4256848784310072
Gradient Descent(6/149): loss=0.42438711010689123
Gradient Descent(7/149): loss=0.42336588329337765
Gradient Descent(8/149): loss=0.42247793064362
Gradient Descent(9/149): loss=0.42166474885918614
Gradient Descent(10/149): loss=0.4209012609262828
Gradient Descent(11/149): loss=0.4201757982356884
Gradient Descent(12/149): loss=0.4194822226418834
Gradient Descent(13/149): loss=0.41881678618858426
Gradient Descent(14/149): loss=0.4181768554963824
Gradient Descent(15/149): loss=0.4175603780927164
Gradient Descent(16/149): loss=0.41696564830374794
Gradient Descent(17/149): loss=0.41639119662856905
Gradient Descent(18/149): loss=0.4158357316212337
Gradient Descent(19/149): loss=0.41529810520044763
Gradient Descent(2

In [ ]:
# Least squares regression using normal equation
weight, loss = least_squares(y, tX)
loss

In [ ]:
# Ridge regression using normal equations
lambda_ = 1e-3
weight, loss = ridge_regression(y, tX, lambda_)
loss

## Generate predictions and save ouput in csv format for submission:

In [6]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [10]:
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)